In [1]:
from clef.utils.data_loading import load_datasets

root_path = '../'

train, dev = load_datasets(preprocess=False,
                           add_author_name=False,
                           add_author_bio=False,
                           root_path= root_path,)


loaded 96 training json lines and 32 dev json lines.


In [2]:
def get_tweet_links(dataset):
    links = []
    for tweet in dataset:
        for account, id, _ in tweet['timeline']:
            links.append(f'{account.strip()}/status/{id}')
    return list(set(links))

In [3]:
links = get_tweet_links(dev)
len(links)

4580

In [4]:
links

['https://twitter.com/nbntweets/status/1589235766194208770',
 'https://twitter.com/nbntweets/status/1590242859936264192',
 'https://twitter.com/moiegy/status/1579091862031597569',
 'https://twitter.com/Pm_govly_ar/status/1559525493178867719',
 'https://twitter.com/mediasrca/status/1341358531182813186',
 'https://twitter.com/LebarmyOfficial/status/1292508637089366016',
 'https://twitter.com/LebarmyOfficial/status/1291771658777305093',
 'https://twitter.com/SecMedCell/status/1576190427786772485',
 'https://twitter.com/CAFCLCC_ar/status/1397477617700417537',
 'https://twitter.com/UNNewsArabic/status/1396973534522511364',
 'https://twitter.com/SerajSat/status/1426452162540883969',
 'https://twitter.com/nbntweets/status/1591008953478959104',
 'https://twitter.com/nbntweets/status/1590623312929230848',
 'https://twitter.com/BBCArabic/status/1408078850312085504',
 'https://twitter.com/nbntweets/status/1589283917705932800',
 'https://twitter.com/UNEgypt/status/1335280903287939073',
 'https://t

automation

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
from tqdm.auto import tqdm
from datetime import datetime
import csv
import time

In [10]:
# setup Selenium
impl_wait_time = 5

driver = webdriver.Chrome()
driver.implicitly_wait(impl_wait_time) # set default waiting strategy

driver.get("https://twitter.com/login")
# could log in to account manually now...

In [9]:

def get_tweet_date(driver: webdriver.Chrome, url: str):
    gpt_4_cutoff = datetime(2023, 4, 1)
    
    try:
        driver.get(url)

        # Find the <time> element containing the datetime attribute
        date_element = driver.find_element(By.XPATH, '//time')

        # Extracting the datetime attribute
        tweet_date = date_element.get_attribute('datetime')

        if tweet_date:
            tweet_datetime = datetime.strptime(tweet_date, '%Y-%m-%dT%H:%M:%S.%fZ')
            return (url, tweet_date, tweet_datetime < gpt_4_cutoff)
        else:
            return (url, None, None)
    except NoSuchElementException:
        return (url, None, None)

In [11]:
with open('./data/tweet_date_check.csv', 'a', newline='') as file:
    writer = csv.writer(file)

    for link in tqdm(links):
        url, dt, before_cutoff = get_tweet_date(driver, link)
        print([url, dt, before_cutoff])
        writer.writerow([url, dt, before_cutoff])
        time.sleep(10)

  0%|          | 0/4580 [00:00<?, ?it/s]

['https://twitter.com/nbntweets/status/1589235766194208770', '2022-11-06T12:38:27.000Z', True]
['https://twitter.com/nbntweets/status/1590242859936264192', '2022-11-09T07:20:17.000Z', True]
['https://twitter.com/moiegy/status/1579091862031597569', '2022-10-09T12:50:12.000Z', True]
['https://twitter.com/Pm_govly_ar/status/1559525493178867719', '2022-08-16T13:00:26.000Z', True]
['https://twitter.com/mediasrca/status/1341358531182813186', '2020-12-22T12:22:49.000Z', True]
['https://twitter.com/LebarmyOfficial/status/1292508637089366016', '2020-08-09T17:10:47.000Z', True]
['https://twitter.com/LebarmyOfficial/status/1291771658777305093', '2020-08-07T16:22:18.000Z', True]
['https://twitter.com/SecMedCell/status/1576190427786772485', '2022-10-01T12:40:56.000Z', True]
['https://twitter.com/CAFCLCC_ar/status/1397477617700417537', '2021-05-26T09:00:02.000Z', True]
['https://twitter.com/UNNewsArabic/status/1396973534522511364', '2021-05-24T23:36:59.000Z', True]
['https://twitter.com/SerajSat/sta

KeyboardInterrupt: 

In [16]:
url, dt, before_cutoff = get_tweet_date(driver, 'https://twitter.com/EFA/status/1312131324040232960')
url, dt, before_cutoff

('https://twitter.com/EFA/status/1312131324040232960',
 '2020-10-02T20:44:20.000Z',
 True)

True
